In [ ]:
using Plots
using LinearAlgebra
using Revise
using LatticeQM

# Tutorial: Tight-binding operators

## Simple 2D lattice: nearest neighbor example

A simple nearest neighbor Hamiltonian
\begin{align}
    H = t \sum_{\langle i,j \rangle} c^\dagger_i c_j
\end{align}

In [ ]:
lat = Geometries2D.honeycomb()

function t(r1, r2=0.0)
    δr=r1.-r2
    
    if 0.9 < norm(δr[1:3])<1.1 # nearest neighbor found!
        return -1.0
    end
    
    return 0.0
end

T = TightBinding.gethops(lat, t)

### Bloch Hamiltonian $H(\vec{k})$

In [ ]:
hk = TightBinding.getbloch(T)
hk([0.1,0.0])

### Band structure

In [ ]:
ks = kpath(lat; num_points=200)
bands = getbands(T, ks)

In [ ]:
plot(bands; size=(300,200))

### Shorter version with pre-defined operators

In [ ]:
T = Hops() # empty hops
Operators.nearestneighbor!(T, lat) # fill in nearest neighbors
Operators.addchemicalpotential!(T, lat, r->(r[4]==0) ? 0.25 : -0.25) # sublattice imbalance

In [ ]:
ks = kpath(lat; num_points=200)
bands = getbands(T, ks)

plot(bands; size=(300,200))

Calculating expectation values:

In [ ]:
T = Hops() # empty hops
Operators.nearestneighbor!(T, lat) # fill in nearest neighbors
Operators.addsublatticeimbalance!(T,lat, 0.5) # shortcut for sublattice imbalance

T = TightBinding.addspin(T, :spinhalf)
Operators.addzeeman!(T, lat, 0.2)

# Additional operators
sz = Operators.spin(lat, "sz")
valley = Operators.valley(lat; spinhalf=true)

In [ ]:
ks = kpath(lat; num_points=200)
bands = getbands(T, ks, [sz, valley])

plot(bands; size=(320,200), colorbar=true, colorbar_title="spin")

In [ ]:
plot(bands, 2; size=(320,200), colorbar=true, colorbar_title="valley")

## Ribbons

### Armchair

In [ ]:
N = 12
lat = Structure.reduceto1D(Geometries2D.honeycomb(), [[1, 1] [N, -N]])
p1 = plot(lat, "sublattice"; supercell=[16], markersize=2)

hops = Operators.graphene(lat)

ks = kpath(lat; num_points=200)
bands = getbands(hops, ks)

p2 = plot(bands; size=(320,200), colorbar=true)

plot(p1,p2, size=(650,200))

### Zigzag 

In [ ]:
N = 12
lat = Structure.reduceto1D(Geometries2D.honeycomb(), [[1, 0] [0, N]])
p1 = plot(lat, "sublattice"; supercell=[16], markersize=2)

hops = Operators.graphene(lat)

ks = kpath(lat; num_points=200)
bands = getbands(hops, ks)

p2 = plot(bands; size=(320,200), colorbar=true)

plot(p1,p2, size=(650,200))